# 🧪 Notebook 1: Test de Analizadores Individuales

Este notebook permite probar cada analizador por separado para entender cómo funcionan.

## Setup Inicial

In [1]:
# Imports
import pandas as pd
import numpy as np
import sys
import json
from pathlib import Path

# Agregar ruta del proyecto
sys.path.append('..')

from fase3_evaluator.analyzers import (
    analyze_completeness,
    analyze_typology,
    analyze_semantic,
    analyze_geospatial,
    analyze_anonymization,
    analyze_ml_readiness
)

print("✅ Imports exitosos")

✅ Imports exitosos


## Crear Dataset de Prueba

Generamos un dataset sintético con problemas típicos

In [2]:
# Dataset de prueba con problemas típicos
df = pd.DataFrame({
    'id': range(1, 21),
    'fecha_evento': pd.date_range('2024-01-01', periods=20, freq='W'),
    'edad': [25, 30, None, 45, 150, 52, 38, None, 60, 28, 
             35, 42, 29, None, 55, 48, 33, 41, 37, 44],
    'sexo': ['M', 'F', 'M', 'M', 'F', 'F', 'M', 'F', 'M', 'F',
             'M', 'F', 'M', 'M', 'F', 'M', 'F', 'M', 'F', 'M'],
    'metodo': ['ahorcamiento', 'intoxicacion', None, 'arma fuego', 'ahorcamiento',
               'precipitacion', 'ahorcadura', 'intoxicacion', 'arma de fuego', None,
               'ahorcamiento', 'sobredosis', 'colgamiento', 'arma fuego', 'intoxicacion',
               'ahorcamiento', 'precipitacion', 'disparo', 'envenenamiento', 'ahorcamiento'],
    'municipio': ['Madrid', 'Valencia', 'Madrid', None, 'Barcelona',
                  'Sevilla', 'Madrid', 'Valencia', 'Barcelona', 'Madrid',
                  'Valencia', 'Madrid', None, 'Barcelona', 'Valencia',
                  'Madrid', 'Sevilla', 'Barcelona', 'Madrid', 'Valencia'],
    'latitud': [40.4168, 39.4699, 40.4168, None, 41.3851,
                37.3891, 40.4168, 39.4699, 41.3851, 40.4168,
                39.4699, 40.4168, None, 41.3851, 39.4699,
                40.4168, 37.3891, 41.3851, 40.4168, 39.4699],
    'longitud': [-3.7038, -0.3763, -3.7038, None, 2.1734,
                 -5.9845, -3.7038, -0.3763, 2.1734, -3.7038,
                 -0.3763, -3.7038, None, 2.1734, -0.3763,
                 -3.7038, -5.9845, 2.1734, -3.7038, -0.3763],
    'tipo_evento': ['consumado', 'intento', 'consumado', 'consumado', 'intento',
                    'consumado', 'intento', 'intento', 'consumado', 'intento',
                    'consumado', 'intento', 'consumado', 'consumado', 'intento',
                    'consumado', 'intento', 'consumado', 'intento', 'consumado']
})

print(f"Dataset creado: {len(df)} registros, {len(df.columns)} columnas")
df.head()

Dataset creado: 20 registros, 9 columnas


,id,fecha_evento,edad,sexo,metodo,municipio,latitud,longitud,tipo_evento
0,1,2024-01-07,25.0,M,ahorcamiento,Madrid,40.4168,-3.7038,consumado
1,2,2024-01-14,30.0,F,intoxicacion,Valencia,39.4699,-0.3763,intento
2,3,2024-01-21,NaN,M,None,Madrid,40.4168,-3.7038,consumado
3,4,2024-01-28,45.0,M,arma fuego,None,NaN,NaN,consumado
4,5,2024-02-04,150.0,F,ahorcamiento,Barcelona,41.3851,2.1734,intento


## 1️⃣ Test: Analizador de Completitud

In [3]:
# Ejecutar análisis de completitud
result_completeness = analyze_completeness(df)

# Mostrar resumen
print("=" * 60)
print("RESUMEN DE COMPLETITUD")
print("=" * 60)
print(f"Total de celdas: {result_completeness['summary']['total_cells']}")
print(f"Celdas faltantes: {result_completeness['summary']['missing_cells']}")
print(f"Porcentaje faltante: {result_completeness['summary']['missing_percentage']:.2f}%")
print(f"\nScore: {result_completeness['evaluation']['score']:.1f}/100")
print(f"Nivel: {result_completeness['evaluation']['level']}")
print(f"\nCampos críticos faltantes: {result_completeness['critical_fields_missing']}")

# Top columnas con más faltantes
print("\n📊 Top columnas con valores faltantes:")
for col_info in result_completeness['top_missing_columns'][:3]:
    print(f"  - {col_info['column']}: {col_info['missing_rate']*100:.1f}%")

RESUMEN DE COMPLETITUD
Total de celdas: 180
Celdas faltantes: 11
Porcentaje faltante: 6.11%

Score: 93.9/100
Nivel: excelente

Campos críticos faltantes: []

📊 Top columnas con valores faltantes:
  - edad: 15.0%
  - metodo: 10.0%
  - municipio: 10.0%


## 2️⃣ Test: Analizador de Tipología

In [4]:
# Ejecutar análisis de tipología
result_typology = analyze_typology(df)

print("=" * 60)
print("ANÁLISIS DE TIPOLOGÍA")
print("=" * 60)
print(f"Total columnas: {result_typology['summary']['total_columns']}")
print(f"Inconsistencias: {result_typology['summary']['inconsistencies_count']}")
print(f"Problemas de encoding: {result_typology['summary']['encoding_issues_count']}")
print(f"\nScore de calidad: {result_typology['summary']['quality_score']:.1f}/100")

# Mostrar inconsistencias detectadas
if result_typology['inconsistencies']:
    print("\n⚠️ Inconsistencias detectadas:")
    for issue in result_typology['inconsistencies'][:3]:
        print(f"  - Columna '{issue['column']}':")
        print(f"    Esperado: {issue['expected']}")
        print(f"    Encontrado: {issue['found']}")
        print(f"    Tasa de inconsistencia: {issue['inconsistency_rate']*100:.1f}%")

ANÁLISIS DE TIPOLOGÍA
Total columnas: 9
Inconsistencias: 1
Problemas de encoding: 0

Score de calidad: 95.6/100

⚠️ Inconsistencias detectadas:
  - Columna 'sexo':
    Esperado: string_categorical
    Encontrado: ['string_categorical', 'string_boolean']
    Tasa de inconsistencia: 45.0%


## 3️⃣ Test: Analizador Semántico

In [5]:
# Ejecutar análisis semántico
result_semantic = analyze_semantic(df)

print("=" * 60)
print("ANÁLISIS SEMÁNTICO")
print("=" * 60)
print(f"Total de issues: {result_semantic['summary']['total_issues']}")
print(f"Issues críticos: {result_semantic['summary']['critical_issues']}")
print(f"Score: {result_semantic['summary']['score']:.1f}/100")
print(f"Nivel de calidad: {result_semantic['summary']['quality_level']}")

# Edades inválidas
if result_semantic['edad_invalida']:
    print(f"\n⚠️ Edades inválidas detectadas: {len(result_semantic['edad_invalida'])}")
    for issue in result_semantic['edad_invalida'][:3]:
        print(f"  - Fila {issue['row']}: edad={issue['value']} ({issue['issue']})")

# Métodos no estandarizados
if result_semantic['metodos_no_estandarizados']:
    print(f"\n📋 Métodos no estandarizados: {len(result_semantic['metodos_no_estandarizados'])}")
    for method in result_semantic['metodos_no_estandarizados'][:5]:
        print(f"  - '{method['value']}' (n={method['count']}) → sugerir: '{method['suggestion']}'")

ANÁLISIS SEMÁNTICO
Total de issues: 7
Issues críticos: 1
Score: 76.0/100
Nivel de calidad: bueno

⚠️ Edades inválidas detectadas: 1
  - Fila 4: edad=150.0 (Edad superior a 120 años)

📋 Métodos no estandarizados: 6
  - 'arma fuego' (n=2) → sugerir: 'arma de fuego'
  - 'ahorcadura' (n=1) → sugerir: 'ahorcamiento'
  - 'sobredosis' (n=1) → sugerir: 'intoxicacion'
  - 'colgamiento' (n=1) → sugerir: 'ahorcamiento'
  - 'disparo' (n=1) → sugerir: 'arma de fuego'


## 4️⃣ Test: Analizador Geoespacial

In [6]:
# Ejecutar análisis geoespacial
result_geospatial = analyze_geospatial(df)

print("=" * 60)
print("ANÁLISIS GEOESPACIAL")
print("=" * 60)
print(f"Geocodificable: {'Sí' if result_geospatial['summary']['geocodable'] else 'No'}")
print(f"Cobertura: {result_geospatial['summary']['coverage']*100:.1f}%")
print(f"Calidad: {result_geospatial['summary']['quality']}")
print(f"Score: {result_geospatial['summary']['score']:.1f}/100")
print(f"Método: {result_geospatial['summary']['primary_method']}")

# Clustering
print(f"\n🗺️ Clustering espacial:")
print(f"  Factible: {'Sí' if result_geospatial['clustering_potential']['feasible'] else 'No'}")
if result_geospatial['clustering_potential']['feasible']:
    print(f"  Algoritmos recomendados: {', '.join(result_geospatial['clustering_potential']['recommended_algorithms'])}")

# Coordenadas
if result_geospatial['coordinates_analysis']:
    coords = result_geospatial['coordinates_analysis']
    print(f"\n📍 Coordenadas:")
    print(f"  Pares válidos: {coords['valid_pairs']}/{coords['total_records']}")
    print(f"  Cobertura: {coords['coverage']*100:.1f}%")

ANÁLISIS GEOESPACIAL
Geocodificable: Sí
Cobertura: 90.0%
Calidad: excelente
Score: 90.0/100
Método: coordenadas_directas

🗺️ Clustering espacial:
  Factible: No

📍 Coordenadas:
  Pares válidos: 18/20
  Cobertura: 90.0%


## 5️⃣ Test: Analizador de Anonimización

In [7]:
# Ejecutar análisis de anonimización
result_anonymization = analyze_anonymization(df)

print("=" * 60)
print("ANÁLISIS DE ANONIMIZACIÓN")
print("=" * 60)
print(f"PII detectada: {'Sí' if result_anonymization['summary']['pii_detected'] else 'No'}")
print(f"Risk score: {result_anonymization['risk_assessment']['score']:.1f}/10")
print(f"Nivel de riesgo: {result_anonymization['risk_assessment']['level']}")
print(f"Crítico: {'Sí' if result_anonymization['risk_assessment']['critical'] else 'No'}")

if result_anonymization['entities_found']:
    print(f"\n⚠️ Entidades PII encontradas:")
    for entity in result_anonymization['entities_found']:
        print(f"  - {entity['type']} en '{entity['column']}': {entity['count']} instancias")
        print(f"    Contribución al riesgo: {entity['risk_contribution']:.1f}")

ANÁLISIS DE ANONIMIZACIÓN
PII detectada: Sí
Risk score: 3.3/10
Nivel de riesgo: moderado
Crítico: No

⚠️ Entidades PII encontradas:
  - ID_NUMBER en 'id': 20 instancias
    Contribución al riesgo: 3.0


## 6️⃣ Test: Analizador ML Readiness

In [8]:
# Ejecutar análisis ML
result_ml = analyze_ml_readiness(df)

print("=" * 60)
print("ANÁLISIS ML READINESS")
print("=" * 60)
print(f"ML viable: {'Sí' if result_ml['ml_viability']['viable'] else 'No'}")
print(f"Confianza: {result_ml['ml_viability']['confidence']}")
print(f"Score: {result_ml['ml_viability']['score']:.1f}/100")

# Target detectado
print(f"\nTarget detectado: {result_ml['target_column']}")

# Features
print(f"\n📊 Features:")
print(f"  Total features potenciales: {result_ml['features_analysis']['potential_features']}")
print(f"  Features usables: {result_ml['features_analysis']['usable_features']}")
print(f"  Numéricas: {len(result_ml['features_analysis']['numeric_features'])}")
print(f"  Categóricas: {len(result_ml['features_analysis']['categorical_features'])}")

# Balance de clases
if result_ml['balance_analysis']:
    balance = result_ml['balance_analysis']
    print(f"\n⚖️ Balance de clases:")
    print(f"  Variable: {balance['column']}")
    print(f"  Número de clases: {balance['n_classes']}")
    print(f"  Nivel de balance: {balance['balance_level']}")
    print(f"  Distribución: {balance['class_distribution']}")

# Leakage risks
if result_ml['leakage_risks']:
    print(f"\n⚠️ Riesgos de leakage detectados: {len(result_ml['leakage_risks'])}")
    for risk in result_ml['leakage_risks'][:3]:
        print(f"  - Columna '{risk['column']}': {risk['reason']}")

# Sugerencias de modelos
print(f"\n🤖 Modelos sugeridos:")
for suggestion in result_ml['model_suggestions']:
    print(f"  - {suggestion['task']}: {', '.join(suggestion['models'])}")
    print(f"    Razón: {suggestion['reason']}")

ANÁLISIS ML READINESS
ML viable: No
Confianza: nula
Score: 50.0/100

Target detectado: tipo_evento

📊 Features:
  Total features potenciales: 6
  Features usables: 6
  Numéricas: 3
  Categóricas: 2

⚖️ Balance de clases:
  Variable: tipo_evento
  Número de clases: 2
  Nivel de balance: balanceado
  Distribución: {'consumado': 11, 'intento': 9}

🤖 Modelos sugeridos:
  - clasificacion_binaria: Logistic Regression, Random Forest, XGBoost
    Razón: Dataset pequeño - modelos interpretables recomendados


## 💾 Guardar Resultados

Opcional: guarda los resultados en JSON para análisis posterior

In [10]:
# CELDA NUEVA: Función helper para convertir tipos NumPy
import numpy as np

def convert_numpy_types(obj):
    """Convierte tipos NumPy a tipos nativos Python para JSON serialization"""
    if isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32, np.int16, np.int8)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32, np.float16)):
        return float(obj)
    elif isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    else:
        return obj

print("Funcion de conversion cargada")

Funcion de conversion cargada


In [11]:
# Guardar resultados consolidados
import json
from pathlib import Path

output_path = Path('../data/outputs/test_analizadores_results.json')
output_path.parent.mkdir(parents=True, exist_ok=True)

# CONVERTIR ANTES DE GUARDAR
all_results_clean = convert_numpy_types(all_results)

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(all_results_clean, f, indent=2, ensure_ascii=False)

print(f"Resultados guardados en: {output_path}")

Resultados guardados en: ../data/outputs/test_analizadores_results.json
